In [1]:
import sys
import os
from os.path import join
from pathlib import Path
sys.path.append(join(Path(os.getcwd()).parent, 'lib'))

In [2]:
sys.path

['',
 '/home/ubuntu/src/cntk/bindings/python',
 '/home/ubuntu/data/code/lib',
 '/home/ubuntu/data/code/lib/fastai/courses/ml1',
 '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip',
 '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6',
 '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload',
 '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages',
 '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions',
 '/home/ubuntu/.ipython',
 '/home/ubuntu/data/code/ref_fdetector/src/lib']

In [3]:
! python3 -m pip install unidecode

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
%matplotlib inline

import json
import os
import pandas as pd
import re
import numpy as np
import pickle
import collections
from importlib import reload
from  mainlib import fasorial, lista, sdict, mm, mm_sep
from unidecode import unidecode
from of_proc import *
from os.path import join
from os import listdir as ld
from unidecode import unidecode
from matplotlib import pyplot as plt

Using TensorFlow backend.


# Informações

In [5]:
def dk(d,k=0): return list(d.values())[k]

In [6]:
def read_info(path = '../../info/info.json'):
    global PATH, OUTPUT_PATH, PROCESSED_PATH, PERFIS_PATH, HORARIOS_PATH, PATH_MM, PATH_FAS, \
    DEG, MOR, PER, FAS, OUTPUT_HEADER,FAS_CONCAT, MM_CONCAT, PICKLES_CONCAT, OUT_INDEX, COLS_LISTA
    with open(path) as f:
        info = json.load(f)
        PATH_FAS = info['paths']['fasoriais2']['input']
        PERFIS_PATH = info['paths']['perfis']['perfis']
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PATH_MM = info['paths']['mm']['input']
        DEG = info['paths']['nn_models']['degrau']
        MOR = info['paths']['nn_models']['morro']
        PER = info['paths']['nn_models']['perfil']
        FAS = info['paths']['nn_models']['fasorial']
        COLS_LISTA = info['lista2']['columns']  
        OUTPUT_HEADER = info['output']['header']
        OUT_INDEX = info['output']['header']
read_info()
OUT = '../outputs/all/'
DATA_DIR = '/home/ubuntu/data/data/maio_2019/setembro/'
LISTA = join(DATA_DIR,'lista-cad-pnt-20190820.csv')

In [7]:
#hc
PATH_MM='/home/ubuntu/data/data/maio_2019/setembro/mm/csv_proc/#pd_mmproc_comp.csv'

In [8]:
global df_mm_new
mmm = mm()
df_mm_new = pd.read_csv(PATH_MM,date_parser = mmm.dateparser, index_col=[0], sep=';')
dummy_mm = ""

## l_______________________________________________________________________

### Carregar modelos neurais

# Coleções principais

In [9]:
global LISTA_CAD
global PERFIL, DEGRAU, MORRO, NN_FAS
global files_fas, files_mm
global HORARIOS, PERFIS, ATIVS, DPERFIS, DPERFIS_NORMAL, DPERFIS_TROCA
global dim, dmi, df_fas, dfiles_mm, dfiles_fas_troca, dfiles_mm_troca, dfiles_fas_normal, dfiles_mm_normal
global o_i_normal, o_i_troca, o_instas, list_i_normal, list_i_troca, list_intas

#### Modelos neurais

In [10]:
#hc
PER = '../../models/r2-DS110k-bs64-c11/r2-DS110k-bs64-c11'
DEG = '../../models/degrau/degrau'
MOR = '../../models/anomalia-1_1_v2/anomalia-1_1_v2'
FAS = '../../models/rn_hc_and/rn_hc_and'

In [11]:
PERFIL, DEGRAU, MORRO, NN_FAS = load_models(PER, DEG, MOR, FAS)

Instructions for updating:
Colocations handled automatically by placer.


## l_______________________________________________________________________

In [12]:
#hc
LISTAF = '/home/ubuntu/data/data/maio_2019/setembro/lista-cad-pnt-20190820-faltantes.csv'

In [13]:
LISTA_CAD = get_list()
#LISTA_CAD.df =  pd.read_csv(LISTA,names = COLS_LISTA)
LISTA_CAD.df =  pd.read_csv(LISTAF,names = COLS_LISTA)
HORARIOS, PERFIS, ATIVS = get_horario_info2()
dim, dmi = dicts(LISTA_CAD)mycol

#### Estruturas auxiliares

In [14]:
o_i_normal = collections.OrderedDict(sorted(LISTA_CAD.mes_days(troca=False).items()))
o_i_troca = collections.OrderedDict(sorted(LISTA_CAD.mes_days(troca=True).items()))
o_instas =  collections.OrderedDict(sorted({**o_i_normal, **o_i_troca}.items()))

#### Arquivos

In [15]:
import A_base
reload(A_base)

<module 'A_base' from '/home/ubuntu/data/code/ref_fdetector/src/lib/A_base.py'>

In [16]:
import A_base as ab

In [17]:
dfiles_fas, nf_fas= get_files_4(list(o_instas.keys()), dim,  PATH_FAS)
ddirs_fas = {k:join(PATH_FAS,v) for k,v in dfiles_fas.items()}

#### Lista , perfis e horários

In [18]:
# Dicionário de instalacao para horario de funcionamento
func =  insta2horario()

In [20]:
# Dicionário de perfis para clusters
perfunc = per2cluster(func,PERFIS)

In [21]:
percats = PERFIS['cat'].fillna(value=0).apply(int).to_dict()

 l___________________________________________________________________________________________________________________________________________

#### Pickles

## l_______________________________________________________________________

In [24]:
! python3 -m pip install pymongo

    100% |################################| 450kB 10.9MB/s 
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [29]:
import pymongo

In [42]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["cas"]

In [43]:
mycol = mydb['fas']

In [46]:
mydict = { "name": "John", "address": "Highway 37" }

x = mycol.insert_one(mydict)

In [47]:
print(mydb.list_collection_names())

['fas']


In [53]:
mycol.find_one()

{'_id': ObjectId('5d853061192a4f302c73fdb7'),
 'name': 'John',
 'address': 'Highway 37'}

In [54]:
for x in mycol.find():
    print(x)

{'_id': ObjectId('5d853061192a4f302c73fdb7'), 'name': 'John', 'address': 'Highway 37'}


In [58]:
myquery = { "address": "Highway 37" }
mydoc = mycol.find(myquery)

In [59]:
for x in mydoc:
  print(x)

{'_id': ObjectId('5d853061192a4f302c73fdb7'), 'name': 'John', 'address': 'Highway 37'}


In [61]:
myquery = { "address": { "$gt": "G" } }

mydoc = mycol.find(myquery)

for x in mydoc:
  print(x)

{'_id': ObjectId('5d853061192a4f302c73fdb7'), 'name': 'John', 'address': 'Highway 37'}


In [62]:
myquery = { "address": {"$regex": "^S"} }

x = mycol.delete_many(myquery)

print(x.deleted_count, " documents deleted.")

0  documents deleted.


## l_______________________________________________________________________

In [156]:
for i in f1.columns:
    mycol[i].insert_one(f1[i])

TypeError: 'Collection' object does not support item assignment

## l_______________________________________________________________________

In [158]:
dp = dframe().dateparser

In [64]:
fpath =  '/home/ubuntu/data/data/maio_2019/setembro/fas-sep/'

In [70]:
fi1=join(fpath,ld(fpath)[0])

In [159]:
f1 = pd.read_csv(fi1, sep=';', names = fc, index_col = [5], date_parser = dp, parse_dates=[5])

In [160]:
f1.head()

,ponto_medicao,serial,RTC,RTP,KE,IA,IB,IC,AIA,AIB,...,DH,FREQ,SEQ,DEMAND_C1,DEMAND_C2,DEMAND_C3,TOT_C1,TOT_C2,TOT_C3,DEM_INTERVAL
data,,,,,,,,,,,,,,,,,,,,,
2019-04-01 00:00:08,BTI400024830,8027241,400/5,1/1,4/10,",07",0,0,308,120,...,0,60,132,6,7,0,10126854,1522269,3319,15
2019-04-01 00:00:08,BTI400024830,8027241,400/5,1/1,4/10,",07",0,0,308,120,...,0,60,132,6,7,0,10126854,1522269,3319,15
2019-04-01 00:00:08,BTI400024830,8027241,400/5,1/1,4/10,",07",0,0,308,120,...,0,60,132,6,7,0,10126854,1522269,3319,15
2019-04-01 00:00:08,BTI400024830,8027241,400/5,1/1,4/10,",07",0,0,308,120,...,0,60,132,6,7,0,10126854,1522269,3319,15
2019-04-01 00:00:08,BTI400024830,8027241,400/5,1/1,4/10,",07",0,0,308,120,...,0,60,132,6,7,0,10126854,1522269,3319,15


In [80]:
from mainlib import dframe
d = dframe().data
fc=d['fasorial']['columns']

In [152]:
import of_proc
reload(of_proc)
from of_proc import *

In [163]:
f1.iloc[:,5:]=f1.iloc[:,5:].applymap(point2comma)

In [173]:
f1.columns

Index(['ponto_medicao', 'serial', 'RTC', 'RTP', 'KE', 'IA', 'IB', 'IC', 'AIA',
       'AIB', 'AIC', 'VA', 'VB', 'VC', 'AVA', 'AVB', 'AVC', 'VAB', 'VBC',
       'VAC', 'AVAB', 'AVBC', 'AVAC', 'PA', 'PB', 'PC', 'FPA', 'FPB', 'FPC',
       'DH', 'FREQ', 'SEQ', 'DEMAND_C1', 'DEMAND_C2', 'DEMAND_C3', 'TOT_C1',
       'TOT_C2', 'TOT_C3', 'DEM_INTERVAL'],
      dtype='object')

In [177]:
d2= f1.loc[:,'IA':'FPC'].to_dict()

In [174]:
d1 = {
'ponto_medicao' : 'BTI400024830',
'serial':  8027241,
'RTC' : '400/5',
'RTP' : '1/1',
'KE' : '4/10',
'TOT_C1' : 10126854,
'TOT_C2' : 1522269,
'TOT_C3': 3319,
'DEM_INTERVAL' :  15,
'DH': 0,
'FREQ':60,
'SEQ':132
}

In [178]:
d2.update(d1)

## l_______________________________________________________________________

In [181]:
mycol.insert_one(d2)

InvalidDocument: documents must have only string keys, key was Timestamp('2019-04-01 00:00:08')